In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **DATA PRE PROCESSING**

In [ ]:
dataset = pd.read_csv('/kaggle/input/dataset1/unpaddy.csv.txt',index_col=[0])

In [ ]:
dataset

In [ ]:
X = dataset.iloc[:, :-1]
Y= dataset.iloc[:,-1]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)


In [ ]:
y_test

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
scaler.fit_transform(X_train)

In [ ]:
x_train_scaled = scaler.transform(X_train)
x_test_scaled = scaler.transform(X_test)

In [ ]:
pd.DataFrame(x_train_scaled)

# **CREATION DU RESEAU DE NEURONE PROFOND**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = keras.Sequential([ 
    layers.Dense(units=512, activation='relu', input_shape=[6]),
    layers.Dropout(0.2),
    layers.Dense(units=256, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(units=512, activation='relu'),
    layers.Dense(units=1, activation="linear")
    # une seule sortie pour la regression 
    
])

In [ ]:
model.compile(
optimizer='adam',
loss='mse',
metrics=['mae']
)
model.summary()

In [ ]:
history = model.fit(x_train_scaled,y_train,batch_size=64,epochs=1000,verbose=1,validation_split=0.2)

# **AFFICHAGE summary CHANGEMENT DE PERTE ET DE LA PRECISION**

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[5:, ['loss','val_loss']].plot();
history_df['loss'].min()

# **PREDICTION DES RESULTATS**

In [ ]:
predictions = model.predict(x_test_scaled)

randomlist = []
for i in range(0,8):
    n = np.random.randint(0,X_test.shape[0])
    randomlist.append(n)
print(randomlist)

In [ ]:
print('les données prédits par notre modele:')
for x in randomlist:
    print(predictions[x])
    

In [ ]:
print('les donnée réel:')
for x in randomlist:
     print(y_test.iloc[x])

**SAUVGARDER LE MODELE**

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test_scaled, y_test)
mse_neural,mae_neural=results
print("l'erreure moyenne quadratique(MSE),l'erreur moyenne absolue(MAE):", results)

In [ ]:
model.save('/kaggle/working/model0') 

# **Random forrest**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

modelf = RandomForestRegressor(n_estimators = 10)
jk=modelf.fit(x_train_scaled, y_train)

y_pred_RF = modelf.predict(x_test_scaled)

mse_RF = mean_squared_error(y_test, y_pred_RF)
mae_RF = mean_absolute_error(y_test, y_pred_RF)
print('l\'erreure moyenne quadratique(MSE) du Random Forest: ', mse_RF)
print('l\'erreure moyenne absolue(MAE) du Random Forest:', mae_RF)

# **nural network**

In [ ]:
### Decision tree
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor()
tree.fit(x_train_scaled, y_train)
y_pred_tree = tree.predict(x_test_scaled)
mse_dt = mean_squared_error(y_test, y_pred_tree)
mae_dt = mean_absolute_error(y_test, y_pred_tree)
print('l\'erreure moyenne quadratique(MSE)  de l\'arbre de decision:  ', mse_dt)
print('l\'erreure moyenne absolue(MAE) de l\'arbre de decision: ', mae_dt)

##############################################

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
MSE=[mse_dt,mse_RF,mse_neural]
MAE=[mae_dt,mae_RF,mae_neural]

In [ ]:
abssices=['Arbre de decission','Foret aléatoire','Réseau de neuronne profond']
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.set_title('MSE des algorithmes de l\'AA')
bar1=ax.bar(abssices,MSE,width=0.35, color='royalblue')
fig.savefig('MSE')

In [ ]:
fig0= plt.figure()
ax2 = fig0.add_axes([0,0,1,1])
ax2.set_title('MAE des algorithmes de l\'AA')
bar2=ax2.bar(abssices,MAE,width=0.35, color='seagreen')
fig0.savefig('MAE')

In [ ]:
import seaborn as sns
sns.displot(dataset['paddy.yield'],kde=True,kind='hist',bins=30)

In [ ]:
sns.pairplot(dataset,palette='coolwarm')

In [ ]:
sns.violinplot(x="paddy.season", y="paddy.yield", data=dataset,palette='rainbow')


In [ ]:
sns.countplot(x='paddy.season',data=dataset)

In [ ]:
sns.barplot(x='paddy.season',y='paddy.yield',data=dataset)

In [ ]:
dataset.corr()

In [ ]:
sns.heatmap(dataset.corr())

In [ ]:
sns.lmplot(x='paddy.slevel',y='paddy.yield',data=dataset,scatter=False)

In [ ]:
sns.lmplot(x='paddy.temp',y='paddy.yield',data=dataset,scatter=False)

In [ ]:
sns.lmplot(x='paddy.ph',y='paddy.yield',data=dataset,scatter=False)

In [ ]:
sns.set()

In [ ]:
figura, axes = plt.subplots(nrows=2,ncols=3,figsize=(12,8))

In [ ]:
sns.regplot(ax=axes[0][0],x='paddy.slevel',y='paddy.yield',data=dataset,scatter=False)
sns.regplot(ax=axes[0][1],x='paddy.temp',y='paddy.yield',data=dataset,scatter=False)
sns.regplot(ax=axes[0][2],x='paddy.ph',y='paddy.yield',data=dataset,scatter=False)
sns.regplot(ax=axes[1][0],x='paddy.season',y='paddy.yield',data=dataset,scatter=False)
sns.regplot(ax=axes[1][1],x='paddy.humidity',y='paddy.yield',data=dataset,scatter=False)
sns.regplot(ax=axes[1][2],x='paddy.variation',y='paddy.yield',data=dataset,scatter=False)
